In [1]:
from lxml import etree
import json
import pandas as pd
import matplotlib.pyplot as plt 
%matplotlib inline  
import seaborn as sns
import regex as reg

In [2]:
import numpy as np
import regex #

# Parsing 

In [3]:
def parse(file_path): 
    i = 0 
    tmp = ''
    key = ''
    xml2 = etree.iterparse(file_path, recover=True)
    data = []
    for action, elem in xml2:
        data.append((elem.attrib, elem.tag, elem.text))
    data_dict = {}
    
    for attrib, tag, text in data: 
        try : 
            tmp = key
            
            key = attrib.get('qualifier')
            element = attrib.get('element')
            
            #way to distinguish eliminate some nan!
            if key == 'none':
                key = element
            
            if key in data_dict.keys() : 
                i = i + 1 
                data_dict[key + str(i)] = text
            else : 
                i = 0 
                data_dict[key] = text
                
        except TypeError: 
            if 'subject' in tag:
                if 'subject' in data_dict.keys():
                    data_dict['subject'].append(text)
                else:
                    data_dict['subject'] = [text]

    return data_dict


### Get all data 

In [4]:
import os
rootdir = '/Users/lguillain/Documents/EPFL2018/Git-semester-project/papers-import/'

In [5]:
all_data = {}
i= 0
for subdir, dirs, files in os.walk(rootdir):
    for file in files:
        path = os.path.join(subdir, file)
        if 'dublin_core' in (path) :
            i += 1
            num_doc = subdir[len(rootdir):]
            all_data[num_doc] = parse(path)

In [6]:
print('got data from {} files'.format(i))

got data from 874 files


In [7]:
df_data = pd.DataFrame(list(all_data.values()), index=all_data.keys())

In [8]:
def convert(arg): 
    try : 
        arg = dateutil.parser.parse(arg)
    except TypeError: 
        arg = arg 
    return arg 

In [9]:
import dateutil.parser
df_data['available'] = df_data['available'].apply(lambda x : convert(x))
df_data['accessioned'] = df_data['accessioned'].apply(lambda x : convert(x))

In [10]:
df_data[df_data['author'].str.contains("Vogel")]

,abstract,accessioned,author,author1,author2,author3,available,citation,iso,issued,...,author17,author18,author19,author20,author21,author22,author23,author24,author25,author26
import2016full/78,Students often have problems formulating und u...,2017-03-21 12:05:42+00:00,"Vogel, Freydis","Kollar, Ingo","Ufer, Stefan","Reiss, Kristina",2017-03-21 12:05:42+00:00,"Vogel, F., Kollar, I., Ufer, S., Reiss, K., & ...",en,2016-07,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
import2015short/373,Argumentation scripts have been proposed as an...,2017-06-19 14:51:54+00:00,"Vogel, Freydis","Kollar, Ingo","Ufer, Stefan","Reichersdorfer, Elisabeth",2017-06-19 14:51:54+00:00,"Vogel, F., Kollar, I., Ufer, S., Reichersdorfe...",en,2015-07,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
df_data.columns

Index([   'abstract', 'accessioned',      'author',     'author1',
           'author2',     'author3',   'available',    'citation',
               'iso',      'issued',   'publisher',       'title',
              'type',         'uri',          None,     'author4',
           'author5',     'author6',     'author7',     'author8',
           'subject',     'author9',    'author10',    'author11',
          'author12',    'author13',    'author14',    'author15',
          'author16',    'author17',    'author18',    'author19',
          'author20',    'author21',    'author22',    'author23',
          'author24',    'author25',    'author26'],
      dtype='object')

# Getting all authors in to one dataframe:

In [12]:
cleaning = df_data.reset_index().melt(id_vars=['index','subject', 'iso', 'uri','type','publisher','title', 'issued', 'accessioned', 'citation', 'available', 'abstract'])
cleaning = cleaning[cleaning.value.notna()]
cleaning = cleaning[cleaning.variable.notna()]

cleaning = cleaning[cleaning.value.map(lambda x: len(x) > 2)]

cleaning['author_name_length'] = cleaning.value.map(lambda x: len(x))

In [13]:
cleaning = cleaning[cleaning.value.notna()]

In [14]:
cleaning['index'].map(lambda x: '2016' in x).sum()

654

In [15]:
import re
cleaning['author_order'] = cleaning.variable.map(lambda x: 0 if len(re.search('\d*$', x).group(0)) == 0 else int(re.search('\d*$', x).group(0)))

In [16]:
del cleaning['variable']

In [17]:
cleaning.head()

,index,subject,iso,uri,type,publisher,title,issued,accessioned,citation,available,abstract,value,author_name_length,author_order
0,import2016full/61,NaN,en,info:doi/10.22318/icls2016.61,Book chapter,Singapore: International Society of the Learni...,Exposing Piaget’s Scheme: Empirical Evidence f...,2016-07,2017-03-21 12:05:42+00:00,"Abrahamson, D., Shayan, S., Bakker, A., & van ...",2017-03-21 12:05:42+00:00,The combination of two methodological resource...,"Abrahamson, Dor",15,0
1,import2016full/95,NaN,en,info:doi/10.22318/icls2016.95,Book chapter,Singapore: International Society of the Learni...,Secondary Teachers’ Emergent Understanding of ...,2016-07,2017-03-21 12:05:42+00:00,"Sandoval, W. A., Kawasaki, J., Cournoyer, N., ...",2017-03-21 12:05:42+00:00,Abstract: The Next Generation Science Standard...,"Sandoval, William A.",20,0
2,import2016full/59,NaN,en,info:doi/10.22318/icls2016.59,Book chapter,Singapore: International Society of the Learni...,Blending Play and Inquiry in Augmented Reality...,2016-07,2017-03-21 12:05:42+00:00,"DeLiema, D., Saleh, A., Lee, C., Enyedy, N., D...",2017-03-21 12:05:42+00:00,Researchers have increasingly demonstrated how...,"DeLiema, David",14,0
3,import2016full/92,NaN,en,info:doi/10.22318/icls2016.92,Book chapter,Singapore: International Society of the Learni...,Making Sense of Making Waves: Co-constructing ...,2016-07,2017-03-21 12:05:42+00:00,"Hardy, L. & White, T. (2016). Making Sense of ...",2017-03-21 12:05:42+00:00,In this paper we argue that collaborative lear...,"Hardy, Lisa",11,0
4,import2016full/66,NaN,en,info:doi/10.22318/icls2016.66,Book chapter,Singapore: International Society of the Learni...,The Effects of Coaching on the Teaching and Le...,2016-07,2017-03-21 12:05:42+00:00,"Raval, H., Kaul, C., & McKenney, S. (2016). Th...",2017-03-21 12:05:42+00:00,Although English is mandatorily introduced as ...,"Raval, Harini",13,0


# Now parse citation to get the shortened name (which can be matched to refrences)

In [18]:
get_names = r'([\w\-\&]*[\,] [\p{L}\.\ ]+[\&\,]?)'

cleaning.reset_index(drop=True, inplace=True)

cleaning['shortend_names'] = cleaning.citation.map(lambda x: re.match(r'[\S\s]*\(\d{4}\)', x, re.U)\
                                                   .group(0)).map(lambda x: [x.replace(',', '').replace('&', '').rstrip() for x in regex.findall(get_names, x)])

cleaning['shortend_names'] = cleaning.apply(lambda x: x['shortend_names'][x['author_order']], axis=1)

In [19]:
cleaning.iloc[845].citation

'Viswanathan, S. A. & Vanlehn, K. (2017). High Accuracy Detection of Collaboration From Log Data and Superficial Speech Features In Smith, B. K., Borge, M., Mercier, E., and Lim, K. Y. (Eds.). (2017). Making a Difference: Prioritizing Equity and Access in CSCL, 12th International Conference on Computer Supported Collaborative Learning (CSCL) 2017, Volume 1. Philadelphia, PA: International Society of the Learning Sciences.'

In [20]:
cleaning[cleaning['index'] == 'import2018/219']

,index,subject,iso,uri,type,publisher,title,issued,accessioned,citation,available,abstract,value,author_name_length,author_order,shortend_names
399,import2018/219,"[boundary crossing, quantified self, infograph...",en,https://doi.dx.org/10.22318/cscl2018.1001,Book chapter,International Society of the Learning Science...,From Quantified Self to Building a More Fit Co...,2018-07,2018-11-04 23:26:23+00:00,"sommer, s. & Polman, J. L. (2018). From Quanti...",2018-11-04 23:26:23+00:00,This design case study considers one teacher's...,"sommer, stephen",15,0,sommer s.
1238,import2018/219,"[boundary crossing, quantified self, infograph...",en,https://doi.dx.org/10.22318/cscl2018.1001,Book chapter,International Society of the Learning Science...,From Quantified Self to Building a More Fit Co...,2018-07,2018-11-04 23:26:23+00:00,"sommer, s. & Polman, J. L. (2018). From Quanti...",2018-11-04 23:26:23+00:00,This design case study considers one teacher's...,"Polman, Joseph L",16,1,Polman J. L.


In [21]:
cleaning.rename(columns={'index': 'file'}, inplace=True)

Extracting the identifier string:

In [22]:
def get_authors_month(sentence, debug = False):
    regex = r'[ééüş\xad\p{L}\,\ \.\:\;\/\&\-\'\`\(\)\’\–\¨\…\‐\*\´\＆\\]*\([\,\ \p{L}\d\-]*(18|19|20)\d{2}[\,\ \p{L}\d\-]*\)'
    match_bad_year = r'[\S\s]*\((18|19|20)\d{2}\/(18|19|20)\d{2}\)'

    match_press = r'[\S\s]*\((i|I)n (P|p)ress|manuscript under review\)'
    match_forth = r'[\S\s]*\((f|F)orthcoming\)'
    match_accepted = r'[\S\s]*\((a|A)ccepted\)'
    match_submitted = r'[\S\s]*\((s|S)ubmitted\)'
    match_underreview = r'[\S\s]*\((u|U)nder (R|r)eview\)'

    #sentence = sentence.lower()
    if reg.match(regex, sentence):
        s = reg.search(regex, sentence).group(0)
        if len(s) > 9:
            return s
    elif re.match(match_bad_year, sentence):
        return re.search(match_bad_year, sentence).group(0)
    elif re.match(match_press, sentence):
        return re.search(match_press, sentence).group(0)
    elif re.match(match_forth, sentence):
        return re.search(match_forth, sentence).group(0)
    elif re.match(match_accepted, sentence):
        return re.search(match_accepted, sentence).group(0)
    elif re.match(match_submitted, sentence):
        return re.search(match_submitted, sentence).group(0)
    elif re.match(match_underreview, sentence):
        return re.search(match_underreview, sentence).group(0)

    return np.nan


def author_title(x):
    """Gets author and tite part of reference string"""
    ref = x
    authors = get_authors_month(x)  
    if isinstance(authors, float):
        return None
    
    search = len(authors)+1

    end = re.search('\.|\?|In Looi', ref[search:])
    if end:
        end = end.start()
    else:
        end = 0
    return ref[: (search+end)]

In [23]:
cleaning['identifier'] = cleaning[cleaning.citation.notna()].citation.map(lambda x: author_title(x))

In [24]:
cleaning.to_csv('../data/Parsed_metadata.csv')

In [25]:
cleaning['merge_index'] = cleaning.apply(lambda x: x['file']+str(x['author_order']), axis=1)